In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
train_dir = 'Train_Set_Folder'
val_dir = 'Validation_Set_Folder'
test_dir = 'Test_Set_Folder'

# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 23972 images belonging to 30 classes.
Found 3030 images belonging to 30 classes.
Found 2998 images belonging to 30 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [5]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=1
)


749/749 ━━━━━━━━━━━━━━━━━━━━ 1838s 2s/step - accuracy: 0.1588 - loss: 2.9695 - val_accuracy: 0.4033 - val_loss: 1.9586


In [6]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print(f'Test accuracy: {test_acc}')


93/93 ━━━━━━━━━━━━━━━━━━━━ 40s 429ms/step - accuracy: 0.4036 - loss: 1.9747
Test accuracy: 0.3884408473968506


In [7]:
import numpy as np
import cv2
import os
from tensorflow.keras.preprocessing import image
from datetime import datetime

# Create 'runs' folder if it doesn't exist
if not os.path.exists('runs'):
    os.makedirs('runs')

def predict_image_and_save(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    img_array_expanded /= 255.

    prediction = model.predict(img_array_expanded)
    class_index = np.argmax(prediction, axis=1)
    class_label = train_generator.class_indices
    class_label = {v: k for k, v in class_label.items()}
    predicted_class = class_label[class_index[0]]

    # Draw bounding box around the plant
    img_with_box = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    height, width, _ = img_with_box.shape
    cv2.rectangle(img_with_box, (0, 0), (width, height), (0, 255, 0), 2)
    label_text = f'{predicted_class} ({prediction[0][class_index[0]]:.2f})'
    cv2.putText(img_with_box, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Save the processed image to the 'runs' folder
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_path = os.path.join('runs', f'{predicted_class}_{timestamp}.jpg')
    cv2.imwrite(output_path, img_with_box)

    return predicted_class, output_path

# Example usage
img_path = 'aug_0_235.jpg'
predicted_class, saved_image_path = predict_image_and_save(img_path)
print(f'The predicted class is: {predicted_class}')
print(f'The image with bounding box is saved at: {saved_image_path}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 519ms/step
The predicted class is: curcuma
The image with bounding box is saved at: runs\curcuma_20240531_000040.jpg
